# AT1 Bond Valuation
© 2025 Marek Ozana

This notebook is a small, opinionated workflow for *relative value* screening in the AT1 universe using **polars-bloomberg** and interactive **Altair** charts.

We pull a bond universe (IDs, tickers, ratings, duration, back-end spread, z-spread) and then visualize how **z-spread** behaves as a function of:
- **Back-end spread** (BE) and
- **Duration** (Dur)

Both views include a simple fit. Bonds **above** the trend line are “cheap” on that axis (wider than the model would suggest), and bonds **below** the line are “rich”.

The two scatter plots are linked: clicking a point in one chart highlights the same bond in the other view, making it easy to spot names that look cheap (or rich) in *both* dimensions.


In [1]:
import json
from pathlib import Path

import altair as alt

alt.renderers.set_embed_options(actions=False)
chart_file = Path("docs/examples/credit") / "at1-valuation-hero.json"
with open(chart_file) as f:
    spec = json.load(f)
alt.Chart.from_dict(spec)

alt.HConcatChart(...)

In [2]:
# Gett bond data from BQL
from polars_bloomberg import BQuery

query = """
let(
    #dur       = duration(duration_type=MODIFIED).value;
    #rtg_bb    = bb_composite().value;
    #zspread   = spread(spread_type=Z).value;
    #be        = flt_spread().value;
)
get(ticker, #dur, #rtg_bb, #zspread, #be)
for(
    filter(
        bondsUniv(types='active', consolidateDuplicates=true),
        ticker() in ['SHBASS', 'SEB', 'SWEDA', 'DANBNK', 'NDAFH', 'DNBNO']
        and crncy() == 'USD'
        and basel_iii_designation() == 'Additional Tier 1'
    )
)
"""

with BQuery() as bq:
    df = bq.bql(query).combine()
df.head(3)

ID,ticker,#dur,#rtg_bb,#zspread,#be
str,str,f64,str,f64,f64
"""YR472577 Corp""","""DANBNK""",3.509404,"""BBB-""",231.091119,259.9
"""BR069680 Corp""","""SWEDA""",2.937217,"""BBB""",270.448417,286.4
"""BP504077 Corp""","""DANBNK""",0.394512,"""BBB-""",154.481971,338.7


In [4]:
import altair as alt

# --- interaction (named, attached once) ---
bond_sel = alt.selection_point(
    name="bond_sel", fields=["ticker"], on="click", empty=False, nearest=True
)
is_selected = {"param": "bond_sel", "empty": False}

# --- shared encodings/styles ---
y_z = alt.Y("#zspread:Q", title="Z-spread (bps)").scale(zero=False)

tooltips = [
    "ID:N",
    "ticker:N",
    "#rtg_bb:N",
    alt.Tooltip("#be:Q", format=",.1f", title="BE"),
    alt.Tooltip("#dur:Q", format=",.2f", title="Dur"),
    alt.Tooltip("#zspread:Q", format=",.1f", title="Z"),
]

base = alt.Chart(df).encode(
    y=y_z,
    color=alt.Color("ticker:N").title("ticker").scale(scheme="paired"),
    tooltip=tooltips,
)


def panel(*, x_field: str, x_title: str, title: str) -> alt.LayerChart:
    """Create a panel with scatter points, regression fit, and labels."""
    x = alt.X(f"{x_field}:Q", title=x_title).scale(zero=False)

    points = base.mark_circle().encode(
        x=x,
        opacity=alt.condition(is_selected, alt.value(1.0), alt.value(0.5)),
        size=alt.condition(is_selected, alt.value(160), alt.value(60)),
    )

    labels = base.mark_text(align="center", baseline="bottom", opacity=0.5).encode(
        x=x,
        text="ticker:N",
        size=alt.condition(is_selected, alt.value(14), alt.value(12)),
    )

    fit = (
        alt.Chart(df)
        .transform_regression(x_field, "#zspread", method="log")
        .mark_line()
        .encode(x=x, y=y_z)
    )

    return (points + fit + labels).properties(title=title, width=280, height=350)


left = panel(x_field="#be", x_title="Back End Spread (bps)", title="Z-spread vs BE")
right = panel(x_field="#dur", x_title="Duration (years)", title="Z-spread vs Duration")

chart = alt.hconcat(left, right).add_params(bond_sel)
# Save the chart
chart.save(chart_file)
chart.save(chart_file.with_suffix(".png"), scale_factor=0.75)